In [156]:
import tensorflow as tf
import pathlib
import os
import time

In [157]:
# Setup path
originalDatadir = pathlib.Path("/Volumes/T7/coco/val2017")
destinationDatadir = "/Volumes/T7/coco/recolorize/color"
destinationDatadir_bw = "/Volumes/T7/coco/recolorize/bw"

In [158]:
# List image in directory
def list_image(path: pathlib.Path):
    imglist = path.glob("*.jpg")
    return [str(img_path) for img_path in imglist]


In [159]:
# a bit dirty but it works
def resize_img(path: str):
    file = tf.io.read_file(path)
    image = tf.image.decode_jpeg(file, channels=3)
    shape = tf.shape(image)
    w, h = shape[0], shape[1]
    img = tf.image.resize_with_crop_or_pad(image=image, target_height=256, target_width=256)
    bwimg = tf.image.rgb_to_grayscale(img)
    resized_jpg = tf.io.encode_jpeg(img, optimize_size=True)
    resizedbw_jpg = tf.io.encode_jpeg(bwimg, optimize_size=True)
    return resized_jpg, resizedbw_jpg

In [160]:
start = time.time()
for img in list_image(originalDatadir):
    # get filename
    colorpath = os.path.join(destinationDatadir, os.path.basename(img))
    bwpath = os.path.join(destinationDatadir_bw, os.path.basename(img))

    # resize and convert to bw
    resized, resizedbw = resize_img(img)

    # save
    tf.io.write_file(bwpath, resized)
    tf.io.write_file(colorpath, resizedbw)

print("Elapsed : {}s".format(time.time() - start))


Elapsed : 60.877880811691284s
